In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
import numpy as np
import pandas as pd
import time
import urllib
import datetime


In [ ]:
'''
取得測站資訊
'''
def get_existing_station_info():
    '''
    取得還有在用的測站
    '''
    url = 'https://e-service.cwb.gov.tw/wdps/obs/state.htm#existing_station'

    station_info_df = pd.read_html(url, match = '1988/07/01', index_col = '站號')
    return station_info_df[0]

def get_revoked_station_info():
    '''
    取得已被撤銷的測站
    '''
    url = 'https://e-service.cwb.gov.tw/wdps/obs/state.htm#existing_station'

    station_info_df = pd.read_html(url, match = '466921', index_col = '站號')
    return station_info_df[0]


In [73]:
### 把測站資料爬下來

# existing_station = get_existing_station_info()
# revoked_station = get_revoked_station_info()

# existing_station.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/existing_station.csv')
# revoked_station.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/revoked_station.csv')

In [132]:
### 把放假資訊爬下來
typhoon = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/typhoon_1.csv', index_col = 0)
typhoon

,ty_name,route,alert_period,intensity,hpa,wind_speed,7_radius,10_radius,alert_num,break
0,彩雲,--,"2021-06-03,2021-06-04,",輕度,NaN,NaN,NaN,--,NaN,NaN
1,閃電,5,"2020-11-05,2020-11-06,2020-11-07,",輕度,982.0,28.0,120.0,--,19,NaN
2,巴威,--,"2020-08-22,",輕度,950.0,43.0,220.0,80,--,NaN
3,米克拉,--,"2020-08-10,2020-08-11,",輕度,992.0,26.0,120.0,--,--,NaN
4,哈格比,--,"2020-08-02,2020-08-03,",中度,975.0,36.0,100.0,30,--,NaN
...,...,...,...,...,...,...,...,...,...,...
106,海燕,--,"2001-10-15,2001-10-16,",中度,965.0,35.0,250.0,100,14,NaN
107,利奇馬,4,"2001-09-23,2001-09-24,2001-09-25,2001-09-26,20...",中度,965.0,35.0,180.0,50,37,NaN
108,納莉,特殊,"2001-09-08,2001-09-09,2001-09-10,",中度,960.0,40.0,150.0,50,12,NaN
109,納莉,特殊,"2001-09-13,2001-09-14,2001-09-15,2001-09-16,20...",中度,960.0,40.0,150.0,50,52,NaN


In [666]:
def get_dayoff_info():
    '''
    從行政院人事總處取得是否上班上課資料
    '''
    
    DAYOFF_INFO_LIST = 'https://www.dgpa.gov.tw/informationlist?uid=374'  # 行政院人事行政總處查詢天災放假的網址
    DGPA = 'https://www.dgpa.gov.tw/'  # 行政院人事行政總處資料根目錄
    KEYWORD = '是否停止'  # table含有的標題
    CHARACTERS = '1234567890年月日'  # mapping日期
    TABLE_COMBINE = pd.DataFrame()
    STYLE = []
    COUNTY_RENAME = {'縣巿名稱': '縣市名稱'}
    TOTAL_DAYS = 0
    TEXT_ONLY = []

    TEXT_ONLY_PAGES = {
        '17': [5,6,7,8,9],
        '18': [3,4,5,6,7,8],
        '19': [2, 5, 6, 9],
        '20': [i for i in range(0,9)],
    }
    pairs = []

    output = pd.DataFrame()

    for key in list(TEXT_ONLY_PAGES.keys()):
        for e in TEXT_ONLY_PAGES[key]:
            pairs.append((int(key), e))

    try:
        for page_num in range(1,31):
            params = {
                'page': page_num
            }
            count = 0
            r = requests.get(DAYOFF_INFO_LIST, params = params)
            soup = bs(r.text)
            l = soup.find_all('div', {'class':'news-list'})

            for dl in l[0].find_all('dl'):
                if (page_num, count) in pairs:  # text only page
                    href = dl.find('a').get('href')
                    r = requests.get(DGPA+href)
                    soup = bs(r.text)
                    text = soup.find('p', {'class': 'word_p1'}).text
                    text = text.replace('\n','').replace('\r','')
                    TEXT_ONLY.append(text)
                    count += 1
                    continue
                try:  # 若標題沒有htm的link
                    href = dl.find('a').get('href')
                    r = requests.get(DGPA+href)
                    soup = bs(r.text)
                    a_tags = soup.find_all('div', {'class': 'download'})[0].find_all('a')

                    if not 'E' in a_tags[0].text:
                        dayoff_href = a_tags[0].get('href')
                    else:
                        dayoff_href = a_tags[1].get('href')

                    dayoff_table_link = DGPA+dayoff_href

                    if params['page'] <= 8:  # 橘色背景table
                        try:
                            table = pd.read_html(dayoff_table_link,
                                                 match = KEYWORD,
                                                )[0]

                            if '區域' not in table.columns:
                                try:
                                    table.columns = ['縣市名稱', '是否停止辦公上課情形','']
                                    column1 = pd.DataFrame({'區域': [0]*len(table)}, index = table.index)
                                    table = pd.concat([column1, table], axis = 1)
                                    STYLE.append('三格橘色')

                                except:
                                    print('發生錯誤', page_num, count)
                            else:
                                STYLE.append('橘色')
#                                 print('drop成功前', count)
#                                 table = table.drop('', axis = 1)
#                                 print('drop成功後', count)

    #                         table = table.drop('', axis = 1)
                        except:
                            print(dayoff_table_link, page_num, count)


                    elif params['page'] > 8:  # 新綠色背景table
                        r = requests.get(dayoff_table_link)
                        soup = bs(r.text)
                        counties = soup.find_all('table')[1].find_all('td', {'valign': 'center','align': 'middle'})
                        announcements = soup.find_all('table')[1].find_all('td', {'valign': 'center','align': 'left'})
                        data = {
                            '區域': [0]*len(counties),
                            '縣市名稱': [county.text for county in counties], 
                            '是否停止辦公上課情形': [announcement.text for announcement in announcements]
                        }
                        table = pd.DataFrame(data)
                        STYLE.append('新綠')


                except:  # 若標題有htm的link
                    dayoff_href = dl.find('a').get('href')
                    dayoff_table_link = DGPA+dayoff_href
                    table = pd.read_html(dayoff_table_link,
                                         match = KEYWORD,
                                         header = 0
                                        )[0]
                    table.rename(COUNTY_RENAME, axis = 1, inplace=True)
                    table = table.drop(len(table)-1, axis = 0)
                    STYLE.append('舊綠')

                date = ''.join(c for c in dl.find('p').text if c in CHARACTERS)

                y = date[:date.index('年')]
                m = date[date.index('年')+1:date.index('月')]
                d = date[date.index('月')+1:date.index('日')]
                # print(y,m,d)
                yyyy = str(int(y)+1911)
                mm = str(m) if len(m) == 2 else '0'+m
                dd = str(d) if len(d) == 2 else '0'+d
                date = yyyy+'-'+mm+'-'+dd

                table['日期'] = np.array([date]*len(table))
        #         break

                TABLE_COMBINE = pd.concat([TABLE_COMBINE, table],axis = 0)
                count += 1
                TOTAL_DAYS += 1
        #     break
    except Exception as e:
        print(e)
        print('total_count: ', TOTAL_DAYS)
        print('page_num: ', page_num)
        print('count: ', count)
        print('dayoff_href: ', dayoff_href)
        print('dayoff_table_link: ', dayoff_table_link)
    #     output = pd.concat([output, TABLE_COMBINE], axis = 0)
    print('finished')
    print('total_days: ', TOTAL_DAYS)
    print('非表格資料筆數: ', len(TEXT_ONLY))

    return [TABLE_COMBINE, TEXT_ONLY]


In [765]:
'''
取得後的資料做基本清洗：移除invalid資料、補0、確認需要人工校對的比數等
'''

# result = get_dayoff_info()
# result[0] = result[0].drop('Unnamed: 0', axis = 1)
# df = result[0]
# df.index = np.array(range(df.shape[0]))

# for idx in df[df['是否停止辦公上課情形'].isnull()].index:
#      df.loc[idx, '是否停止辦公上課情形'] = df.loc[idx, '是否停止上班上課情形']

# df.drop('是否停止上班上課情形', axis = 1, inplace = True)
# df.drop('', axis = 1, inplace = True)

# for idx in df.index:
#     if len(str(df.loc[idx, '區域'])) > 4:
#         df.drop(idx, axis = 0, inplace = True)

# for idx in df.index:
#     if len(str(df.loc[idx, '縣市名稱'])) >= 4:
#         df.drop(idx, axis = 0, inplace = True)
    
# for idx in df[df['區域'].isnull()].index:
#     df.loc[idx, '區域'] = 0

# count = 0
# for s in df['是否停止辦公上課情形']:
#     if len(str(s))>20:
#         count += 1
# print(count)



575


In [770]:
'''
儲存資料
'''
# df.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/dayoff/dayoff_info.csv', encoding = 'utf-8')

In [ ]:
'''


段落



'''

In [2]:
'''
透過測站id找颱風警報期間的觀測資料
中央氣象局測站資料查詢系統：
https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain
經過query parameters中，中文經過兩次url編碼
# '''
# df_typhoon_flattened = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/typhoon_info/df_typhoon_flattened.csv', index_col = 0)
# df_station = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_info/Stations+Typhoon break_final.csv')
def get_alert_period_observations(df_station, df_typhoon_flattened):
    '''
    以月份取得中央氣象局觀測站的資料
    df_station為Sue整理之包含測站ID、中文名稱、
    '''
    import pandas as pd
    import numpy as np
    import requests
    import urllib
    import time
    start = time.time()
#     day_url = 'https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?
    month_url = 'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?'
    headers = {
        'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'
    }
    
    station_info = df_station[['Station', 'number']]
    
    downloads = {}
    
    error_dicts = {
        'no_data_stations':{}, 
        'no_url_access':{}, 
        'pandas_cant_read':{}, 
        'requests_not_200':{},
    }

    error_dicts_colnames = [
        'station',
        'stname',
        'month',
        'url',
    ]

    for dic in error_dicts.keys():
        for colname in error_dicts_colnames:  # 欲來生成裝不同型態Error的字典
            error_dicts[dic][str(colname)] = []
    
    month_count = 0
    download_count = 0
    requests_count = 0
    read_table_count = 0
    
    for month in df_typhoon_flattened['issue_alert_dates'].str[:7].unique():
        loop_end_time = time.time()
        
        print(f'month_getting: {(month_count+1)}/80')
        print(f'time_through: {loop_end_time - start}')
        month_count += 1
        for idx in station_info.index:
            station = station_info.loc[idx, 'number']
            stname = station_info.loc[idx, 'Station']
            params = {
                'command': 'viewMain',
                'station': station,
                'stname': '' ,
                'datepicker': month,
            }
            url = month_url + urllib.parse.urlencode(params)
#             print(url)
            pickups = [station, stname, month, url]

#             if requests_count > 10:
#                 break
            
            requests_count += 1
            
            try:

                r = requests.get(url, 
#                                  header=headers, 
#                                  timeout=5,
                                )

                try:
                    if r.status_code == 200:  #正確
                        
                        table = pd.read_html(url,
                                             header = 2,
                                             match = 'press',
                                             encoding = 'utf-8'
                                             )[0]
                        read_table_count += 1
                        if table.shape[0] == 0:  # 如果連線後沒有觀測資料，將資料存入invalid_stations中
                            for colname, pickup in zip(error_dicts_colnames, pickups):
                                error_dicts['no_data_stations'][colname].append(pickup) 

                            continue  # 跳過這個loop的後續
                        
                        downloads[month+'_'+station+'_'+stname] = table  # 下載的資料存放
                        download_count += 1

                    else:
                        for colname, pickup in zip(error_dicts_colnames, pickups):
                            error_dicts['requests_not_200'][colname].append(pickup) 
                        continue

                except ValueError:
                    for colname, pickup in zip(error_dicts_colnames, pickups):
                        error_dicts['pandas_cant_read'][colname].append(pickup) 
                    continue

            except:
                for colname, pickup in zip(error_dicts_colnames, pickups):
                    error_dicts['no_url_access'][colname].append(pickup)
                    continue

#     errors = {
#         'no_data_stations': pd.DataFrame(no_data_stations),
#         'no_url_access': pd.DataFrame(no_url_access),
#         'pandas_cant_read': pd.DataFrame(pandas_cant_read),
#         'requests_not_200': pd.DataFrame(requests_not_200),
#     }
    
    class observation_requests():
        def __init__(self, downloads, error_dicts):
            self.downloads = downloads
            self.error_dicts = error_dicts
        def get_no_data_stations(self):
            return pd.DataFrame(self.error_dicts['no_data_stations'])
        def get_no_url_access(self):
            return pd.DataFrame(self.error_dicts['no_url_access'])
        def get_pandas_cant_read(self):
            return pd.DataFrame(self.error_dicts['pandas_cant_read'])
        def get_requests_not_200(self):
            return pd.DataFrame(self.error_dicts['requests_not_200'])
    
    print('month_iter: ', month_count)
    print('download_count', download_count)
    print('requests_count', requests_count)
    print('read_table_count', read_table_count)
    
    end = time.time()
    print(end - start)
    return observation_requests(downloads, error_dicts)


In [3]:
df_typhoon_flattened = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/typhoon_info/df_typhoon_flattened.csv', index_col = 0)
df_station = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_info/Stations+Typhoon break_final.csv')
observation_requests = get_alert_period_observations(df_station = df_station,
                                                    df_typhoon_flattened = df_typhoon_flattened)
# observation_requests

month_getting: 1/80
time_through: 0.012743949890136719
month_getting: 2/80
time_through: 281.32023000717163
month_getting: 3/80
time_through: 542.0159802436829
month_getting: 4/80
time_through: 814.0994720458984
month_getting: 5/80
time_through: 1070.2755870819092
month_getting: 6/80
time_through: 1335.225832939148
month_getting: 7/80
time_through: 1591.1127219200134
month_getting: 8/80
time_through: 1851.938644170761
month_getting: 9/80
time_through: 2120.5250959396362
month_getting: 10/80
time_through: 2385.8739981651306
month_getting: 11/80
time_through: 2643.3375549316406
month_getting: 12/80
time_through: 2896.386474132538
month_getting: 13/80
time_through: 3171.291488170624
month_getting: 14/80
time_through: 3423.8100090026855
month_getting: 15/80
time_through: 3687.6418981552124
month_getting: 16/80
time_through: 3938.861403942108
month_getting: 17/80
time_through: 4186.26860499382
month_getting: 18/80
time_through: 4431.176355838776
month_getting: 19/80
time_through: 4688.09182

In [29]:
import time
dfs = observation_requests.downloads

count = 0
key_list = list(dfs.keys())

DF = pd.DataFrame()
df_1000 = pd.DataFrame()

start1 = time.time()
for key in range(len(key_list)):
    time_loop_start = time.time()
    key = key_list.pop()
    dfs[key]['file_name'] = key
    df_1000 = pd.concat([df_1000, dfs[key]], axis = 0)
    count += 1
    
    if count % 1000 == 0:
        DF = pd.concat([DF, df_1000], axis = 0)
        time_loop_end = time.time()
        df_1000 = pd.DataFrame()
        print(f'loop: {count}, loop_time: {time_loop_end-time_loop_start}')
        
    if count == len(dfs.keys()):
        DF = pd.concat([DF, df_1000], axis = 0)
        time_end = time.time()
        print(f'loop: {count}, loop_time: {time_end-time_loop_start}')

        end1 = time.time()
print(end1-start1)


    

loop: 1000, loop_time: 0.024052143096923828
loop: 2000, loop_time: 0.039093017578125
loop: 3000, loop_time: 0.10047626495361328
loop: 4000, loop_time: 0.09550738334655762
loop: 5000, loop_time: 0.15078496932983398
loop: 6000, loop_time: 0.2835850715637207
loop: 7000, loop_time: 0.22262239456176758
loop: 8000, loop_time: 0.6859989166259766
loop: 9000, loop_time: 0.5630049705505371
loop: 10000, loop_time: 0.23770713806152344
loop: 11000, loop_time: 0.6005949974060059
loop: 12000, loop_time: 0.29785704612731934
loop: 13000, loop_time: 0.3226468563079834
loop: 14000, loop_time: 0.32090187072753906
loop: 15000, loop_time: 0.40442490577697754
loop: 16000, loop_time: 0.6723971366882324
loop: 17000, loop_time: 2.5004849433898926
loop: 18000, loop_time: 0.4900681972503662
loop: 19000, loop_time: 0.42908430099487305
loop: 20000, loop_time: 0.4075620174407959
loop: 21000, loop_time: 0.4210350513458252
loop: 22000, loop_time: 0.4465758800506592
loop: 23000, loop_time: 0.660315752029419
loop: 24000

In [30]:
# DF.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_observation/observations.csv')

In [26]:
# df_error_1 = observation_requests.get_no_data_stations()
# df_error_2 = observation_requests.get_no_url_access()
# df_error_3 = observation_requests.get_pandas_cant_read()
# df_error_4 = observation_requests.get_requests_not_200()

# df_error_1.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_observation/no_data_stations.csv')
# df_error_2.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_observation/no_url_access.csv')
# df_error_3.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_observation/pandas_cant_read.csv')
# df_error_4.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_observation/requests_not_200.csv')
# print('ok')

In [24]:
key_list = list(dfs.keys())
count = 0
for i in range(len(key_list)):
#     key = key_list.pop()
    key = key_list[i]
    count += 1
    if count % 100 == 0:
        print(count)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [22]:
count

0

In [ ]:
### 抓颱風資料

In [84]:
month_url = 'https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?'
params = {
    'command': 'viewMain',
    'station': 'C0AH30',
#     'stname': urllib.parse.quote_plus(urllib.parse.quote_plus('信義')),
    'stname': urllib.parse.quote_plus(''),
    'datepicker': '2022-01',
}
url = month_url + urllib.parse.urlencode(params)

In [85]:
table = pd.read_html(url,
                     header = 2,
                     match = 'press',
                     encoding = 'utf-8'
                     )[0]
table

,ObsTime,StnPres,SeaPres,StnPresMax,StnPresMaxTime,StnPresMin,StnPresMinTime,Temperature,T Max,T Max Time,...,PrecpMax60,PrecpMax60Time,SunShine,SunShineRate,GloblRad,VisbMean,EvapA,UVI Max,UVI Max Time,Cloud Amount
0,1,939.5,...,941.3,2022-01-01 09:13,938.0,2022-01-01 14:32,11.7,13.1,2022-01-01 12:13,...,...,...,...,...,...,...,...,...,...,...
1,2,938.7,...,940.4,2022-01-02 08:51,937.5,2022-01-02 14:54,11.6,12.4,2022-01-02 01:01,...,...,...,...,...,...,...,...,...,...,...
2,3,937.6,...,939.1,2022-01-03 08:03,936.3,2022-01-03 14:11,12.4,14.9,2022-01-03 13:31,...,...,...,...,...,...,...,...,...,...,...
3,4,935.5,...,937.7,2022-01-04 10:58,933.1,2022-01-04 23:53,13.3,15.2,2022-01-04 23:52,...,...,...,...,...,...,...,...,...,...,...
4,5,931.8,...,933.6,2022-01-05 23:33,930.2,2022-01-05 14:11,14.7,15.7,2022-01-05 12:01,...,...,...,...,...,...,...,...,...,...,...
5,6,935.2,...,937.5,2022-01-06 22:41,933.1,2022-01-06 00:13,X,...,...,...,...,...,...,...,...,...,...,...,...,...
6,7,936.5,...,937.5,2022-01-07 13:41,935.0,2022-01-07 14:22,9.8,15.1,2022-01-07 13:39,...,...,...,...,...,...,...,...,...,...,...
7,8,934.4,...,936.3,2022-01-08 09:21,933.0,2022-01-08 14:50,10.9,13.2,2022-01-08 19:21,...,...,...,...,...,...,...,...,...,...,...
8,9,931.8,...,933.3,2022-01-09 00:01,929.9,2022-01-09 14:29,12.2,14.6,2022-01-09 22:42,...,...,...,...,...,...,...,...,...,...,...
9,10,930.7,...,932.5,2022-01-10 08:53,928.9,2022-01-10 14:27,12.4,13.5,2022-01-10 16:41,...,...,...,...,...,...,...,...,...,...,...


In [111]:
import pandas as pd
import urllib
import json
import requests

# 用板橋測站當例子
url_JSON_fullurl = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization=CWB-5393733F-1F8E-4BD2-A358-0360AABEE6EB&format=JSON&locationName=%E6%9D%BF%E6%A9%8B"

# query根目錄
url_root = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?'

# 需要輸入的參數
params = {
    'Authorization': 'CWB-5393733F-1F8E-4BD2-A358-0360AABEE6EB',
    'format': 'JSON',
    'locationName': '板橋',
}

# 將url_root & 參數合併生成網址，會長得跟url_JSON_fullurl一樣
url_JSON = url_root + urllib.parse.urlencode(params)

print('url_JSON_fullurl: ', url_JSON_fullurl, sep = '\n')
print('url_JSON: ',url_JSON, sep = '\n')

r_JSON = requests.get(url_JSON)

# json的巢狀結構
j = json.loads(r_JSON.text)
print('JSON巢狀結構： ')
j

url_JSON_fullurl: 
https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization=CWB-5393733F-1F8E-4BD2-A358-0360AABEE6EB&format=JSON&locationName=%E6%9D%BF%E6%A9%8B
url_JSON: 
https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization=CWB-5393733F-1F8E-4BD2-A358-0360AABEE6EB&format=JSON&locationName=%E6%9D%BF%E6%A9%8B
JSON巢狀結構： 


{'success': 'true',
 'result': {'resource_id': 'O-A0003-001',
  'fields': [{'id': 'lat', 'type': 'Double'},
   {'id': 'lon', 'type': 'Double'},
   {'id': 'locationName', 'type': 'String'},
   {'id': 'stationId', 'type': 'String'},
   {'id': 'obsTime', 'type': 'Timestamp'},
   {'id': 'elementName', 'type': 'String'},
   {'id': 'elementValue', 'type': 'String'},
   {'id': 'parameterName', 'type': 'String'},
   {'id': 'parameterValue', 'type': 'String'}]},
 'records': {'location': [{'lat': '24.999447',
    'lon': '121.433812',
    'locationName': '板橋',
    'stationId': '466880',
    'time': {'obsTime': '2022-02-19 13:30:00'},
    'weatherElement': [{'elementName': 'ELEV', 'elementValue': '11'},
     {'elementName': 'WDIR', 'elementValue': '70'},
     {'elementName': 'WDSD', 'elementValue': '2.30'},
     {'elementName': 'TEMP', 'elementValue': '16.60'},
     {'elementName': 'HUMD', 'elementValue': '0.94'},
     {'elementName': 'PRES', 'elementValue': '1016.60'},
     {'elementName': '24R',

In [112]:
# 用Pandas解析裡面的結構，找到records --> location[0] --> weatherElement是觀測欄位與對應的資料
# 清理資料
ObsJSON = pd.DataFrame(j['records']['location'][0]['weatherElement']).T
ObsJSON.columns = ObsJSON.loc['elementName',:]
ObsJSON = ObsJSON.drop('elementName', axis = 0)
ObsJSON

elementName,ELEV,WDIR,WDSD,TEMP,HUMD,PRES,24R,H_FX,H_XD,H_FXT,...,H_10D,H_F10T,H_UVI,D_TX,D_TXT,D_TN,D_TNT,D_TS,VIS,Weather
elementValue,11,70,2.30,16.60,0.94,1016.60,13,7.50,60,1251,...,80,1254,-99,17.50,0949,16.50,0757,0,3-6,陰有雨


In [107]:
j

{'success': 'true',
 'result': {'resource_id': 'O-A0003-001',
  'fields': [{'id': 'lat', 'type': 'Double'},
   {'id': 'lon', 'type': 'Double'},
   {'id': 'locationName', 'type': 'String'},
   {'id': 'stationId', 'type': 'String'},
   {'id': 'obsTime', 'type': 'Timestamp'},
   {'id': 'elementName', 'type': 'String'},
   {'id': 'elementValue', 'type': 'String'},
   {'id': 'parameterName', 'type': 'String'},
   {'id': 'parameterValue', 'type': 'String'}]},
 'records': {'location': [{'lat': '24.999447',
    'lon': '121.433812',
    'locationName': '板橋',
    'stationId': '466880',
    'time': {'obsTime': '2022-02-19 13:20:00'},
    'weatherElement': [{'elementName': 'ELEV', 'elementValue': '11'},
     {'elementName': 'WDIR', 'elementValue': '70'},
     {'elementName': 'WDSD', 'elementValue': '2.70'},
     {'elementName': 'TEMP', 'elementValue': '16.80'},
     {'elementName': 'HUMD', 'elementValue': '0.93'},
     {'elementName': 'PRES', 'elementValue': '1016.20'},
     {'elementName': '24R',

elementName,ELEV,WDIR,WDSD,TEMP,HUMD,PRES,24R,H_FX,H_XD,H_FXT,...,H_10D,H_F10T,H_UVI,D_TX,D_TXT,D_TN,D_TNT,D_TS,VIS,Weather
elementValue,11,80,2.30,17.20,0.93,1015.90,12,2.50,20,1139,...,80,1134,-99,17.50,0949,16.50,0757,0,>30,陰


In [62]:
j['records']['location']

[{'lat': '24.999447',
  'lon': '121.433812',
  'locationName': '板橋',
  'stationId': '466880',
  'time': {'obsTime': '2022-02-19 12:30:00'},
  'weatherElement': [{'elementName': 'ELEV', 'elementValue': '11'},
   {'elementName': 'WDIR', 'elementValue': '80'},
   {'elementName': 'WDSD', 'elementValue': '2.30'},
   {'elementName': 'TEMP', 'elementValue': '17.20'},
   {'elementName': 'HUMD', 'elementValue': '0.93'},
   {'elementName': 'PRES', 'elementValue': '1015.90'},
   {'elementName': '24R', 'elementValue': '12'},
   {'elementName': 'H_FX', 'elementValue': '2.50'},
   {'elementName': 'H_XD', 'elementValue': '20'},
   {'elementName': 'H_FXT', 'elementValue': '1139'},
   {'elementName': 'H_F10', 'elementValue': '1.40'},
   {'elementName': 'H_10D', 'elementValue': '80'},
   {'elementName': 'H_F10T', 'elementValue': '1134'},
   {'elementName': 'H_UVI', 'elementValue': '-99'},
   {'elementName': 'D_TX', 'elementValue': '17.50'},
   {'elementName': 'D_TXT', 'elementValue': '0949'},
   {'eleme